In [1]:
#this version v4.0 has a correction in the formula used to average the stdev
print('starting')
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from scipy.stats import norm
from scipy.optimize import minimize

import time
import os

from sklearn.metrics import roc_curve, auc


starting


In [2]:
import scipy
scipy.__version__

'1.7.1'

In [3]:
#define paths
best_TSS_path = '../all_joe_feature_matrixes/220906_consensus_best_tss.targeted_panel.csv'
TSS_filter_path = '../all_joe_feature_matrixes/TSS.all_PDX.gene_tracker_thru_clustering.for_ALD_221212.csv'
TSS_path1 = '../../../SCLC_TSS_targeted/analysis/merged_data/set1_cfDNA.TSS.2022-11-22.tsv'
TSS_path2 = '../../../SCLC_TSS_targeted/analysis/merged_data/set2_cfDNA.TSS.2022-11-22.tsv'

TFBS_path = '../motif_aligned/REST_only_aligned_2022-10-21.tsv'

#THIS ONE IS NEW (below)
TFBS_kmeans_path = '../all_joe_feature_matrixes/features_for_integrated_model.TFBS.using_adj_rand_idx.230104__reassign_AS_plus_N.sites_only_for_ALD.csv'

metadata_path = '/fh/fast/ha_g/user/adoebley/data/SCLC_cfDNA_metadata/metadata_table/metadata_from_joe/SCLC_key_metadata_2023-01-05.tsv'
color_path = '/fh/fast/ha_g/user/adoebley/data/SCLC_cfDNA_metadata/metadata_table/metadata_from_joe/221117.color_map.csv'
out_dir = 'results_v4'
table_dir = 'tables_v4'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
if not os.path.exists(table_dir):
    os.makedirs(table_dir)


In [4]:
#define other universal parameters

#starting values for optimization
alphas = [0.1,0.5,0.9]
subtype_fracs = [0.1,0.5,0.9]

TFBS_cols = np.arange(-360,360,15).astype(str)
TSS_cols = np.arange(-60,165,15).astype(str)

print('TFBS', len(TFBS_cols))
print('TSS', len(TSS_cols))

TFBS 48
TSS 15


In [5]:
#define variable parameters that should be recorded when saving outputs
params = {'stdev': False}

In [6]:
#import color dict
color_dict = pd.read_csv(color_path, keep_default_na = False)
color_dict = color_dict.set_index('label')['namedcolor'].to_dict()

In [7]:
#import metadata
def import_metadata(metadata_path):
    metadata = pd.read_csv(metadata_path, sep='\t', keep_default_na = False)
    metadata = metadata.replace('',np.nan)
    metadata = metadata[(metadata['sequencing_type']=='targeted')]
    metadata = metadata[(metadata[['protocol_pass', 'coverage_pass', 'other_pass']]=='pass').all(axis = 1)]

    metadata = metadata.set_index('sample')

    metadata['subtype'] = metadata['subtype'].replace(np.nan,'NA')
    
    #define REST activity
    metadata['REST-activity'] = metadata['REST_status'].replace(np.nan,'NA')
    
    for TF in ['ASCL1','NEUROD1','POU2F3','ATOH1']:
        #new version!
        metadata[TF+'-activity'] = np.where(metadata['subtype'].str.contains(TF),'High','Low')
        metadata[TF+'-activity'] = np.where(metadata['subtype'].isin(['NA','Unknown']),'Unknown',metadata[TF+'-activity'])
        metadata[TF+'-activity'] = np.where(metadata['histology'].isin(['LUAD','LUSC']),'Low',metadata[TF+'-activity'])  

    #drop ATOH1 activity for the MSK samples without ATOH1 data
    metadata['ATOH1-activity'] = np.where((metadata['ATOH1'].isnull()) & (metadata['histology']=='SCLC'), 'Unknown', metadata['ATOH1-activity'])
    
    return(metadata)

metadata = import_metadata(metadata_path)
print('samples:',len(metadata))

samples: 135


In [8]:
metadata[['histology','subtype','ASCL1-activity','NEUROD1-activity','POU2F3-activity','ATOH1-activity']].drop_duplicates()


,histology,subtype,ASCL1-activity,NEUROD1-activity,POU2F3-activity,ATOH1-activity
sample,,,,,,
18_P-0044_sclc_patient_L-0303_v2,SCLC,Unknown,Unknown,Unknown,Unknown,Unknown
FHSC14_P-0092_sclc_pdx_L-0340_v2,SCLC,ASCL1,High,Low,Low,Low
H211_P-0098_sclc_pdx_L-0345_v2,SCLC,POU2F3,Low,Low,High,Low
LC0174_P-0014_benign_patient_L-0311_v2,Non-malign.,NA,Unknown,Unknown,Unknown,Unknown
LC0609_P-0031_nsclc_patient_L-0320_v2,LUAD,NA,Low,Low,Low,Low
LX33B_P-0100_sclc_pdx_L-0347_v2,SCLC,NEUROD1,Low,High,Low,Low
MEDO-PL-0898-001_P-0108_sclc_patient_L-0237_v2,SCLC,Pan-low,Low,Low,Low,Low
MEDO-PL-1722-004_P-0120_sclc_patient_L-0273_v2,SCLC,ATOH1,Low,Low,Low,High
DIVE_CDX17_P-0396_sclc_pdx_L-0569,SCLC,ATOH1+NEUROD1,Low,High,Low,High


In [9]:
for TF in ['ASCL1','NEUROD1','POU2F3','ATOH1','REST']:
    display(metadata[metadata['tumor_fraction']>0.05][['source',TF+'-activity']].value_counts(sort = False, dropna = False))
    print('\n')

source   ASCL1-activity
patient  High              15
         Low               18
         Unknown           46
pdx      High               7
         Low               18
dtype: int64

source   NEUROD1-activity
patient  High                 8
         Low                 25
         Unknown             46
pdx      High                10
         Low                 15
dtype: int64

source   POU2F3-activity
patient  Low                33
         Unknown            46
pdx      High                3
         Low                22
dtype: int64

source   ATOH1-activity
patient  High               1
         Low               26
         Unknown           52
pdx      High               4
         Low               21
dtype: int64

source   REST-activity
patient  High             11
         Low              16
         Unknown          52
pdx      High              8
         Low              17
dtype: int64

In [10]:
#import TFBS
def import_TFBS(TFBS_path):
    TFBS = pd.read_csv(TFBS_path, sep='\t')
    TFBS = TFBS[TFBS['sample'].isin(metadata.index)]
    TFBS = TFBS[['sample', 'unique_ID']+list(TFBS_cols)]
    TFBS = TFBS.set_index(['sample','unique_ID']).unstack()
    TFBS.columns = ['_'.join(m) for m in TFBS.columns]

    TFBS_features = list(TFBS.columns)
    #drop null features
    TFBS_features = TFBS[TFBS_features].columns[~(TFBS[TFBS_features].isnull().any())]
    TFBS = TFBS[TFBS_features].astype(float)
    return(TFBS, TFBS_features)

TFBS, TFBS_features = import_TFBS(TFBS_path)
print('TFBS_features', len(TFBS_features))
print('TFBS',TFBS.shape)

TFBS_features 71774
TFBS (135, 71774)


In [11]:
def import_TSS_list(best_TSS_path,TSS_filter_path):
    best_TSS_per_gene = pd.read_csv(best_TSS_path)
    TSS_filter = pd.read_csv(TSS_filter_path, index_col = 0)

    best_TSS_per_gene = best_TSS_per_gene.rename(columns={'Gene stable ID':'gene_id'})
    best_TSS = TSS_filter.merge(best_TSS_per_gene, on = ['gene_id'])

    best_TSS = best_TSS[best_TSS['defined_griffin_all_pdx']==True]
    best_TSS['new_strand'] = np.where(best_TSS['Strand']==1,'+','-')
    best_TSS['unique_ID'] = best_TSS['gene_symbol_y']+'.'+best_TSS['Chrom']+'.'+best_TSS['TSS'].astype(str)+'.'+best_TSS['new_strand']

    return(best_TSS)

best_TSS = import_TSS_list(best_TSS_path,TSS_filter_path)

print('expected number of TSS', len(best_TSS))
print('expected features', len(best_TSS)*len(TSS_cols))


expected number of TSS 15313
expected features 229695


In [12]:
#import TSS
def import_TSS(TSS_path1,TSS_path2,best_TSS):
    TSS1 = pd.read_csv(TSS_path1, sep='\t')
    gene_name_dict = TSS1[['gene_id','unique_ID']].drop_duplicates().reset_index(drop = True)
    TSS1 = TSS1[['sample', 'unique_ID']+list(TSS_cols)]

    TSS2 = pd.read_csv(TSS_path2, sep='\t')
    TSS2 = TSS2[['sample', 'unique_ID']+list(TSS_cols)]

    TSS = TSS1.append(TSS2, ignore_index = True)
    TSS = TSS[TSS['sample'].isin(metadata.index)]

    del(TSS1,TSS2)

    TSS = TSS[TSS['unique_ID'].isin(best_TSS['unique_ID'])]

    TSS = TSS.set_index(['sample','unique_ID']).unstack()
    TSS.columns = ['_'.join(m) for m in TSS.columns]

    TSS_features = TSS.columns
    
    return(TSS,TSS_features)

TSS,TSS_features = import_TSS(TSS_path1,TSS_path2,best_TSS)

print('TSS shape', TSS.shape)


TSS shape (135, 229695)


In [13]:
#merge feature matrixes and metadata
data = TFBS.merge(TSS, left_index = True, right_index = True)
data = data.merge(metadata, left_index = True, right_index = True)
print(data.shape)

(135, 301509)


In [14]:
#define features
def define_features(TFBS_features,best_TSS):
    feature_dict = {}

    TSS_SCLC_v_NSCLC = best_TSS[(best_TSS['defined_griffin_all_pdx']) &
             (best_TSS['defined_entropy_all_pdx']) &
             (best_TSS['defined_exp_all_pdx']) &
             (best_TSS['exp_var_all_pdx']>1) &
             (best_TSS['kmeans_MW_U_pvalue']<1e-2) &
             (best_TSS['kmeans_grp_diff_exp']>2) &
             (best_TSS['ccle_SvN_frxn_exprsd_chi2_pvalue_adj']<0.1)]
    TSS_SCLC_v_NSCLC_allcols = [m+'_'+n for m in TSS_cols for n in TSS_SCLC_v_NSCLC['unique_ID']]
    #feature_dict['TSS-SCLC-v-NSCLC-allcols'] = TSS_SCLC_v_NSCLC_allcols
    
    
    TSS_SCLCsub = best_TSS[(best_TSS['defined_griffin_all_pdx']) &
                            (best_TSS['defined_entropy_all_pdx']) &
                            (best_TSS['defined_exp_all_pdx']) &
                            (best_TSS['exp_var_all_pdx']>1) &
                            (best_TSS['kmeans_MW_U_pvalue']<1e-2) &
                            (best_TSS['kmeans_grp_diff_exp']>2) &
                            (best_TSS['ccle_sclc_corr_max']>0.6)]
    
    TSS_SCLCsub_allcols = [m+'_'+n for m in TSS_cols for n in TSS_SCLCsub['unique_ID']]
    #feature_dict['TSS-SCLCsub-allcols'] = TSS_SCLCsub_allcols
    
    TFBS_kmeans = pd.read_csv(TFBS_kmeans_path)['unique_ID'].drop_duplicates()

    TFBS_kmeans_allcols = [m+'_'+n for m in TFBS_cols for n in TFBS_kmeans]
    TFBS_kmeans_allcols = pd.Series(TFBS_kmeans_allcols)[pd.Series(TFBS_kmeans_allcols).isin(TFBS_features)]

    #feature_dict['TFBS-kmeans-allcols'] = TFBS_kmeans_allcols
    
    
    feature_dict['TFBS-TSS-SCLC-v-NSCLC-allcols'] = list(TFBS_kmeans_allcols) + list(TSS_SCLC_v_NSCLC_allcols)
    
    #feature_dict['TFBS-TSS-SCLCsub-allcols'] = list(TFBS_kmeans_allcols) + list(TSS_SCLCsub_allcols)
    

    for TF in ['ASCL1','NEUROD1','POU2F3','ATOH1','REST']:
        TSS_SCLCsub_current = TSS_SCLCsub[TSS_SCLCsub['ccle_sclc_corr_w_'+TF]>0.6]['unique_ID']
        TSS_SCLCsub_current_allcols = [m+'_'+n for m in TSS_cols for n in TSS_SCLCsub_current]    
        #feature_dict['TSS-SCLCsub-'+TF+'-allcols'] = TSS_SCLCsub_current_allcols

        TFBS_SCLCsub_current_allcols = list(TFBS_kmeans_allcols[TFBS_kmeans_allcols.str.contains(TF)])
        #feature_dict['TFBS-SCLCsub-'+TF+'-allcols'] = TFBS_SCLCsub_current_allcols
    
        feature_dict['TFBS-TSS-SCLCsub-'+TF+'-allcols'] = TFBS_SCLCsub_current_allcols + TSS_SCLCsub_current_allcols 

    #collect metadata and diplay what is in the feature dict
    feature_metadata = {}
    for key in feature_dict.keys():
        current_feats = [m.split('_')[1] for m in feature_dict[key]]
        current_feats = pd.Series(current_feats).str.split('.',expand = True)
        current_tfbs = current_feats[(current_feats[3].isnull())]
        current_tss = current_feats[~(current_feats[3].isnull())]

        print(key)
        print('\tunique_sites:',len(current_feats.drop_duplicates()), 'total_pos:',len(current_feats))
        print('\tunique_TFBS:',len(current_tfbs.drop_duplicates()), 'total_TFBS_pos:',len(current_tfbs))
        print('\tunique_TSS:',len(current_tss.drop_duplicates()),'total_TSS_pos:',len(current_tss))
    
    
    return(feature_dict)

feature_dict = define_features(TFBS_features,best_TSS)


TFBS-TSS-SCLC-v-NSCLC-allcols
	unique_sites: 755 total_pos: 19739
	unique_TFBS: 277 total_TFBS_pos: 12569
	unique_TSS: 478 total_TSS_pos: 7170
TFBS-TSS-SCLCsub-ASCL1-allcols
	unique_sites: 80 total_pos: 1992
	unique_TFBS: 24 total_TFBS_pos: 1152
	unique_TSS: 56 total_TSS_pos: 840
TFBS-TSS-SCLCsub-NEUROD1-allcols
	unique_sites: 85 total_pos: 3519
	unique_TFBS: 68 total_TFBS_pos: 3264
	unique_TSS: 17 total_TSS_pos: 255
TFBS-TSS-SCLCsub-POU2F3-allcols
	unique_sites: 81 total_pos: 3657
	unique_TFBS: 74 total_TFBS_pos: 3552
	unique_TSS: 7 total_TSS_pos: 105
TFBS-TSS-SCLCsub-ATOH1-allcols
	unique_sites: 5 total_pos: 75
	unique_TFBS: 0 total_TFBS_pos: 0
	unique_TSS: 5 total_TSS_pos: 75
TFBS-TSS-SCLCsub-REST-allcols
	unique_sites: 132 total_pos: 4916
	unique_TFBS: 111 total_TFBS_pos: 4601
	unique_TSS: 21 total_TSS_pos: 315


In [15]:
#collect metadata and diplay what is in the feature dict

def collect_feature_metadata(feature_dict):
    feature_metadata = pd.DataFrame()
    for key in feature_dict.keys():

        current_feats = [m.split('_')[1] for m in feature_dict[key]]
        current_feats = pd.Series(current_feats).str.split('.',expand = True)
        current_tfbs = current_feats[(current_feats[3].isnull())]
        current_tss = current_feats[~(current_feats[3].isnull())]

        current_metadata = pd.Series(dtype = object)
        name_dict = {'TFBS-TSS-SCLC-v-NSCLC-allcols':'SCLC_vs_NSCLC',
                    'TFBS-TSS-SCLCsub-ASCL1-allcols':'ASCL1_activity',
                    'TFBS-TSS-SCLCsub-NEUROD1-allcols':'NEUROD1_activity',
                    'TFBS-TSS-SCLCsub-POU2F3-allcols':'POU2F3_activity',
                    'TFBS-TSS-SCLCsub-ATOH1-allcols':'ATOH_activity',
                    'TFBS-TSS-SCLCsub-REST-allcols':'REST_activity'}
        current_metadata['ANALYSIS'] = name_dict[key]

        TFBS_sites = current_tfbs.drop_duplicates()[0].value_counts()
        TFBS_sites.index = TFBS_sites.index +'_TFBS'
        current_metadata = current_metadata.append(TFBS_sites)

        TFBS_feats = current_tfbs[0].value_counts()
        TFBS_feats.index = TFBS_feats.index +'_TFBS_features'
        current_metadata = current_metadata.append(TFBS_feats)
        current_metadata = current_metadata.sort_index()

        current_metadata['total_TFBS'] = len(current_tfbs.drop_duplicates())
        current_metadata['total_TFBS_features'] = len(current_tfbs)

        current_metadata['total_TSS'] = len(current_tss.drop_duplicates())
        current_metadata['total_TSS_features'] = len(current_tss)

        current_metadata['total_sites'] = len(current_feats.drop_duplicates())
        current_metadata['total_sites_features'] = len(current_feats)


        #current_metadata.name = name_dict[key]
        #current_metadata = current_metadata.copy()
        feature_metadata = feature_metadata.append(current_metadata, sort = True, ignore_index = True)#.copy()

    return(feature_metadata)
feature_metadata = collect_feature_metadata(feature_dict)
feature_metadata.to_csv(table_dir+'/feature_metadata.tsv', sep='\t', index = False)

In [16]:
feature_metadata

,ANALYSIS,ASCL1_TFBS,ASCL1_TFBS_features,NEUROD1_TFBS,NEUROD1_TFBS_features,POU2F3_TFBS,POU2F3_TFBS_features,REST_TFBS,REST_TFBS_features,total_TFBS,total_TFBS_features,total_TSS,total_TSS_features,total_sites,total_sites_features
0,SCLC_vs_NSCLC,24.0,1152.0,68.0,3264.0,74.0,3552.0,111.0,4601.0,277.0,12569.0,478.0,7170.0,755.0,19739.0
1,ASCL1_activity,24.0,1152.0,NaN,NaN,NaN,NaN,NaN,NaN,24.0,1152.0,56.0,840.0,80.0,1992.0
2,NEUROD1_activity,NaN,NaN,68.0,3264.0,NaN,NaN,NaN,NaN,68.0,3264.0,17.0,255.0,85.0,3519.0
3,POU2F3_activity,NaN,NaN,NaN,NaN,74.0,3552.0,NaN,NaN,74.0,3552.0,7.0,105.0,81.0,3657.0
4,ATOH_activity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,5.0,75.0,5.0,75.0
5,REST_activity,NaN,NaN,NaN,NaN,NaN,NaN,111.0,4601.0,111.0,4601.0,21.0,315.0,132.0,4916.0


In [17]:
analysis_dict = {'SCLC-v-NSCLC':
                 {'target_column':'histology',
                 'pos_phenos':['SCLC'],
                 'training_histologies':['SCLC','LUAD','LUSC'],
                 'feature_sets':['TFBS-TSS-SCLC-v-NSCLC-allcols']
                 }}

for TF in ['ASCL1','NEUROD1','POU2F3','ATOH1','REST']:
    analysis_dict[TF] = {'target_column':TF+'-activity',
                 'pos_phenos':['High'],
                 'training_histologies':['SCLC','LCNEC'],
                 'feature_sets':['TFBS-TSS-SCLCsub-'+TF+'-allcols']
                        }

In [18]:
# test = pd.DataFrame([[1,2,3,4,5,8],[1,2,3,4,5,6]]).T


# print(test.std())
# np.sqrt((test.std()**2).mean())

In [19]:
def calc_distributions(current_training,features,variable_std):
    #calculate the training featbest_TSS_paths from the training data
    training_feat_means = current_training.groupby(['a','s'])[features].mean().T
    training_feat_stdev = current_training.groupby(['a','s'])[features].std().T
    
    #if not calculating a standard dev for each group, take the average
    if not variable_std:
        means = np.sqrt((training_feat_stdev**2).mean(axis = 1))
        #means = training_feat_stdev.mean(axis = 1)
        for col in training_feat_stdev.columns:    
            training_feat_stdev[col] = means
            
    return(training_feat_means,training_feat_stdev)

In [20]:
def run_minimizer(alphas,subtype_fracs,test_sample_data,training_feat_means,training_feat_stdev,features):
    sample_results = pd.DataFrame()
    for a_start in alphas:
        for s_start in subtype_fracs:
            init_prob = prob([a_start,s_start],test_sample_data,training_feat_means,training_feat_stdev,features)  
            result = minimize(prob, [a_start,s_start], args = (test_sample_data,training_feat_means,training_feat_stdev,features), bounds = [(0,1),(0,1)], method = 'SLSQP')
            a,s = result['x']
            sample_results = sample_results.append({'sample':test_sample_data.name, 'a_start':a_start,'s_start':s_start, 'a_hat':a,'s_hat':s,'init_prob':init_prob ,'prob':result['fun']}, ignore_index = True)

    return sample_results

In [21]:
def prob(arr,test_sample_data,training_feat_means,training_feat_stdev,features):
    a,s = arr
    mu_mix = (1-a)*training_feat_means[(0,0)] + a*s*training_feat_means[(1,1)] + a*(1-s)*training_feat_means[(1,0)]
    std_mix = np.sqrt((1-a)*(training_feat_stdev[(0,0)]**2) + a*s*(training_feat_stdev[(1,1)]**2) + a*(1-s)*(training_feat_stdev[(1,0)]**2))

    return -np.sum(np.log2(norm.pdf(test_sample_data[features], loc=mu_mix, scale=std_mix)))


In [22]:
for key in analysis_dict.keys():
    print(key)
    target_column = analysis_dict[key]['target_column']
    positive_phenos = analysis_dict[key]['pos_phenos']
    training_histologies = analysis_dict[key]['training_histologies']
    feature_sets = analysis_dict[key]['feature_sets']
    
    training_PDX = data[(data['source']=='pdx') & (data[target_column]!='Unknown') & (data['histology'].isin(training_histologies))]
    training_HD = data[(data['histology']=='Non-malign.')]
    training_data = training_PDX.append(training_HD)
    training_data['a'] = np.where(training_data['source']=='pdx',1,0)
    training_data['s'] = np.where((training_data[target_column].isin(positive_phenos)),1,0)
    
    #test on everything that isn't in the training data
    test_data = data[(data['source']=='patient') & ~(data['histology']=='Non-malign.')].copy()
    #test_data['a'] = test_data['tumor_fraction']
    #test_data['s'] = np.where((test_data[target_column].isin(positive_phenos)),1,0)
    PDX_test = data[~(data.index.isin(training_data.index)) & ~(data.index.isin(test_data.index))]

    for feature_set in feature_sets:
        #define output files
        training_export_name = out_dir+'/'+key+'_'+feature_set+'_training.tsv'
        test_export_name = out_dir+'/'+key+'_'+feature_set+'_test.tsv'
        PDX_test_export_name = out_dir+'/'+key+'_'+feature_set+'_PDX-test.tsv'   
        
        if os.path.exists(training_export_name) and os.path.exists(test_export_name) and os.path.exists(PDX_test_export_name):
            print(feature_set,'complete')
            continue
        
        features = list(feature_dict[feature_set])
        print(feature_set, '\t', len(features))
        
        #run the model
        start_time = time.time()

        #run model on PDX+HD
        training_results = pd.DataFrame()

        for i in range(len(training_data)):
            #get data
            LOO_test = training_data[['a','s']+features].iloc[i]
            LOO_training = training_data[['a','s']+features].drop(index = LOO_test.name)

            #calc mean and std on other models
            training_feat_means,training_feat_stdev = calc_distributions(LOO_training,features,params['stdev'])

            sample_results = run_minimizer(alphas,subtype_fracs,LOO_test,training_feat_means,training_feat_stdev,features)

            training_results = training_results.append(sample_results.sort_values(by = 'prob').iloc[0], ignore_index = True)

            del(LOO_test,LOO_training,training_feat_means,training_feat_stdev,sample_results)
            
        print('training CV time', np.round(time.time()-start_time),'samples:',len(training_results))

        #calc mean and std on full training data set
        training_feat_means,training_feat_stdev = calc_distributions(training_data,features,params['stdev'])

        #run model on patient samples
        test_results = pd.DataFrame()
        for i in range(len(test_data)):
            current_test_patient = test_data[features].iloc[i]
            sample_results = run_minimizer(alphas,subtype_fracs,current_test_patient,training_feat_means,training_feat_stdev,features)
            test_results = test_results.append(sample_results.sort_values(by = 'prob').iloc[0], ignore_index = True)
            del(current_test_patient,sample_results)

        #for each additional PDX model
        PDX_test_results = pd.DataFrame()
        for i in range(len(PDX_test)):
            current_test_patient = PDX_test[features].iloc[i]
            sample_results = run_minimizer(alphas,subtype_fracs,current_test_patient,training_feat_means,training_feat_stdev,features)
            PDX_test_results = PDX_test_results.append(sample_results.sort_values(by = 'prob').iloc[0], ignore_index = True)
            del(current_test_patient,sample_results)

        print('test run time', np.round(time.time()-start_time),'samples:',len(test_results),len(PDX_test_results))
        
        #save results
        training_results.to_csv(training_export_name, sep='\t', index = False)
        test_results.to_csv(test_export_name, sep='\t', index = False)
        PDX_test_results.to_csv(PDX_test_export_name, sep='\t', index = False)

        del(feature_set,features,start_time,training_export_name,test_export_name,PDX_test_export_name)
        del(training_results,test_results,PDX_test_results)
    del(test_data,training_data,PDX_test)
    print('\n')
    

SCLC-v-NSCLC
TFBS-TSS-SCLC-v-NSCLC-allcols 	 19739
training CV time 90.0 samples: 28
test run time 316.0 samples: 106 1


ASCL1
TFBS-TSS-SCLCsub-ASCL1-allcols 	 1992
training CV time 16.0 samples: 24
test run time 86.0 samples: 106 5


NEUROD1
TFBS-TSS-SCLCsub-NEUROD1-allcols 	 3519
training CV time 20.0 samples: 24
test run time 94.0 samples: 106 5


POU2F3
TFBS-TSS-SCLCsub-POU2F3-allcols 	 3657
training CV time 21.0 samples: 24
test run time 89.0 samples: 106 5


ATOH1
TFBS-TSS-SCLCsub-ATOH1-allcols 	 75
training CV time 8.0 samples: 24
test run time 48.0 samples: 106 5


REST
TFBS-TSS-SCLCsub-REST-allcols 	 4916
training CV time 22.0 samples: 24
test run time 118.0 samples: 106 5




In [23]:
print('done')

done
